In [5]:
%pip install opencv-python
%pip install mediapipe
%pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
import cv2

In [7]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    sucesso, frame = cap.read()
    if not sucesso:
        print('Ignorando o frame vazio da câmera')
        continue
    cv2.imshow('Camera', frame)
    if cv2.waitKey(10) & 0xFF == ord('c'):
        break
cap.release()
cv2.destroyAllWindows()   

KeyboardInterrupt: 